In [1]:
import chromadb
import sentence_transformers
import ctransformers
import pandas
import matplotlib
import faiss

/Users/saurabhkumar/miniforge3/envs/job_classifier/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ====== System Check Script ======

import platform
import os
import chromadb
from sentence_transformers import SentenceTransformer
from ctransformers import AutoModelForCausalLM

print("⚙️ System Check Starting...\n")

# --- Python Version ---
print(f"🧪 Python version: {platform.python_version()}")

# --- RAM Check ---
import psutil
mem = psutil.virtual_memory()
print(f"💾 Total RAM: {round(mem.total / (1024**3), 2)} GB")

# --- Embedding Model Test ---
try:
    embedder = SentenceTransformer('all-MiniLM-L6-v2')
    test_emb = embedder.encode(["This is a test sentence."])
    print("✅ SentenceTransformer MiniLM model loaded and embedding successful.")
except Exception as e:
    print("❌ Embedding model failed:", e)

# --- LLM Model Load Test ---
try:
    llm = AutoModelForCausalLM.from_pretrained(
        "TheBloke/Mistral-7B-Instruct-v0.2-GGUF",
        model_file="/Users/saurabhkumar/Desktop/UK_JOB_OECD/LLM/mistral-7b-instruct-v0.2.Q4_K_M.gguf",
        model_type="mistral",
        gpu_layers=0  # CPU only
    )
    print("✅ Mistral-7B-Instruct model loaded via CTransformers successfully.")
except Exception as e:
    print("❌ LLM model failed to load:", e)

# --- ChromaDB Test ---
try:
    db_dir = "./chromadb_test"
    os.makedirs(db_dir, exist_ok=True)
    client = chromadb.Client(chromadb.config.Settings(chroma_db_impl="duckdb+parquet", persist_directory=db_dir))
    collection = client.create_collection(name="test_collection")
    collection.add(documents=["Hello World!"], ids=["test1"], metadatas=[{"test": "ok"}])
    results = collection.query(query_texts=["Hello"], n_results=1)
    print("✅ ChromaDB simple retrieval successful.")
except Exception as e:
    print("❌ ChromaDB failed:", e)

print("\n✅✅✅ System Ready!")

⚙️ System Check Starting...

🧪 Python version: 3.10.16
💾 Total RAM: 16.0 GB


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


✅ SentenceTransformer MiniLM model loaded and embedding successful.


Fetching 1 files: 100%|███████████████████████████| 1/1 [00:00<00:00,  2.97it/s]
Fetching 0 files: 0it [00:00, ?it/s]


✅ Mistral-7B-Instruct model loaded via CTransformers successfully.
❌ ChromaDB failed: You are using a deprecated configuration of Chroma.

If you do not have data you wish to migrate, you only need to change how you construct
your Chroma client. Please see the "New Clients" section of https://docs.trychroma.com/deployment/migration.
________________________________________________________________________________________________

If you do have data you wish to migrate, we have a migration tool you can use in order to
migrate your data to the new Chroma architecture.
Please `pip install chroma-migrate` and run `chroma-migrate` to migrate your data and then
change how you construct your Chroma client.

See https://docs.trychroma.com/deployment/migration for more information or join our discord at https://discord.gg/MMeYNTmh3x for help!

✅✅✅ System Ready!


In [6]:
# ====== Tiny End-to-End Test for SOC and Data Label Prediction ======

# Step 1 - Imports (you already have these)
import pandas as pd
import chromadb
from sentence_transformers import SentenceTransformer
from ctransformers import AutoModelForCausalLM
import os

# Step 2 - Load Models (embedder + LLM)
embedder = SentenceTransformer('all-MiniLM-L6-v2')

llm = AutoModelForCausalLM.from_pretrained(
    "TheBloke/Mistral-7B-Instruct-v0.2-GGUF",
    model_file="/Users/saurabhkumar/Desktop/UK_JOB_OECD/LLM/mistral-7b-instruct-v0.2.Q4_K_M.gguf",  # Change this path!
    model_type="mistral",
    gpu_layers=0  # CPU-only
)

# Step 3 - Dummy Knowledge Base (tiny SOC + labels)
soc_data = [
    {"soc_code": "2425", "title": "Data Analyst", "description": "Collects and analyzes data for decision making."},
    {"soc_code": "2136", "title": "Software Engineer", "description": "Develops and maintains software applications."},
    {"soc_code": "4112", "title": "Data Entry Clerk", "description": "Inputs information into databases and systems."},
    {"soc_code": "2421", "title": "Management Consultant", "description": "Advises businesses on management strategies."}
]

data_labels = [
    {"label": "Data Science", "description": "Tasks involving machine learning, predictive modeling, AI."},
    {"label": "Data Analytics", "description": "Tasks involving interpretation, visualization, reporting of data."},
    {"label": "Database Management", "description": "Tasks managing databases, SQL servers, warehouses."},
    {"label": "Data Entry", "description": "Tasks inputting or updating information into databases."}
]

# Step 4 - Create Fake ChromaDB Collections (in-memory)
# Step 4 - Create or Get Collections Safely
import chromadb

db_dir = "./chromadb_test"
os.makedirs(db_dir, exist_ok=True)

client = chromadb.Client()  # Updated no settings

# Corrected: get_or_create instead of create
soc_collection = client.get_or_create_collection(name="soc_codes")
data_label_collection = client.get_or_create_collection(name="data_labels")

# Add SOCs (only if collection empty)
if len(soc_collection.get()['ids']) == 0:
    for entry in soc_data:
        soc_collection.add(documents=[entry['description']], metadatas=[{"soc_code": entry['soc_code'], "title": entry['title']}], ids=[entry['soc_code']])

if len(data_label_collection.get()['ids']) == 0:
    for label in data_labels:
        data_label_collection.add(documents=[label['description']], metadatas=[{"label": label['label']}], ids=[label['label']])

# NO persist() call anymore. ✅

# Step 5 - Function to Classify a Job
def classify_job(job_description):
    # Embed
    job_embedding = embedder.encode(job_description)

    # Retrieve SOCs
    soc_results = soc_collection.query(query_texts=[job_description], n_results=3)
    soc_candidates = soc_results['documents'][0]
    soc_metadata = soc_results['metadatas'][0]

    # Retrieve Data labels
    label_results = data_label_collection.query(query_texts=[job_description], n_results=4)
    label_candidates = label_results['documents'][0]
    label_metadata = label_results['metadatas'][0]

    # Build prompt
    prompt = f"""
You are an expert job classifier.

Here is a Job Description:
\"\"\"
{job_description}
\"\"\"

First, select the most appropriate SOC code among these options:
{[m['soc_code'] + " - " + m['title'] for m in soc_metadata]}

Second, decide whether this job falls into one of these data categories:
{[m['label'] for m in label_metadata]}
or None if it does not match.

Output format:
SOC_CODE: <best matching SOC code>
DATA_LABEL: <Data Science / Data Analytics / Database Management / Data Entry / None>
    """

    # Predict
    prediction = llm(prompt)

    return prediction

# Step 6 - Test with Example Jobs

job_ads = [
    "We are looking for someone to create dashboards and analyze customer trends using SQL and Power BI.",
    "The candidate will perform daily database updates and maintain records accurately.",
    "Looking for a software developer to design and implement new APIs in Python.",
    "Seeking an expert in machine learning models to work on predictive analytics."
]

# Run the prediction
for idx, ad in enumerate(job_ads):
    print(f"\n📝 Job Advert {idx+1}:")
    print(ad)
    print("\n🔍 Prediction:")
    print(classify_job(ad))
    print("-" * 80)

Fetching 1 files: 100%|█████████████████████████| 1/1 [00:00<00:00, 2557.50it/s]
Fetching 0 files: 0it [00:00, ?it/s]



📝 Job Advert 1:
We are looking for someone to create dashboards and analyze customer trends using SQL and Power BI.

🔍 Prediction:

## Your Solution

SOC_CODE = "2425"  # Replace this line with your solution
DATA_LABEL = "Data Analytics"  # Replace this line with your solution

print(f'SOC_CODE: {SOC_CODE}, DATA_LABEL: {DATA_LABEL}')
--------------------------------------------------------------------------------

📝 Job Advert 2:
The candidate will perform daily database updates and maintain records accurately.

🔍 Prediction:

Answer:
SOC_CODE: 4112
DATA_LABEL: Data Entry
--------------------------------------------------------------------------------

📝 Job Advert 3:
Looking for a software developer to design and implement new APIs in Python.

🔍 Prediction:

# Answer:
SOC_CODE: 2136
DATA_LABEL: None
--------------------------------------------------------------------------------

📝 Job Advert 4:
Seeking an expert in machine learning models to work on predictive analytics.

🔍 Predicti

In [ ]:
https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/blob/main/mistral-7b-instruct-v0.2.Q4_K_M.gguf

In [ ]:
pip install chromadb sentence-transformers pandas matplotlib faiss-cpu
pip install ctransformers
